In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("./sales-data/Amazon Sale Report.csv")

C:\Users\peric\AppData\Local\Temp\ipykernel_10944\469468274.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./sales-data/Amazon Sale Report.csv")


In [6]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [7]:
df['Amount_EUR'] = df['Amount']*0.011

In [8]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Crear una copia del dataframe original
df = df.copy()

# Duplicar datos hacia atrás hasta 2020
for años_atras in range(1, 3):  # 1 año atrás (2021), 2 años atrás (2020)
    nuevo_df = df.copy()
    nuevo_df['Date'] = nuevo_df['Date'] - pd.DateOffset(years=años_atras)

    # Reducir 'Qty' en 1 o 2 unidades si es mayor a 0
    nuevo_df['Qty'] = nuevo_df['Qty'].apply(lambda x: max(0, x - np.random.randint(1, 3)) if x > 0 else x)

    # Reducir 'Amount' y 'Amount_EUR' en un 10-15%
    factor_reduccion = np.random.uniform(0.85, 0.90)  # Reduce entre 10-15%
    for col in ['Amount', 'Amount_EUR']:
        if col in nuevo_df.columns:
            nuevo_df[col] = nuevo_df[col] * factor_reduccion

    # Agregar los datos generados al dataset expandido
    df = pd.concat([df, nuevo_df], ignore_index=True)

# Guardar el nuevo dataset con datos hasta 2020
# df_expanded.to_excel("dataset_expandido_hasta_2020.xlsx", index=False)

C:\Users\peric\AppData\Local\Temp\ipykernel_10944\3939793250.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


Agregar metricas que faltaban 
@NICO falta hacer que estas metricas afecten al df con los datos historicos

In [9]:
def hash_asin(asin):
    return abs(hash(asin)) % 11  # Genera un valor entre 0 y 10

# 📌 Calcular COGS Unitario (35% - 45% del precio unitario)
df["COGS Unit"] = ((df["Amount_EUR"] / df["Qty"]) * (0.35 + (df["ASIN"].apply(hash_asin) / 100))).round(2)

# 📌 Calcular COGS Total
df["COGS"] = df["COGS Unit"] * df["Qty"]

# 📌 Calcular Other Expenses (15% - 25% del Amount_EUR)
df["Other Expenses"] = (df["Amount_EUR"] * (0.15 + (df["ASIN"].apply(hash_asin) / 100))).round(2)

# 📌 Calcular Gross Profit
df["Gross Profit"] = df["Amount_EUR"] - df["COGS"]

# 📌 Calcular Net Profit
df["Net Profit"] = df["Gross Profit"] - df["Other Expenses"]

In [10]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22', 'Amount_EUR', 'COGS Unit', 'COGS',
       'Other Expenses', 'Gross Profit', 'Net Profit'],
      dtype='object')

In [11]:
df.describe()

c:\Users\peric\anaconda3\envs\sanisi-env\Lib\site-packages\numpy\lib\_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,index,Date,Qty,Amount,ship-postal-code,Amount_EUR,COGS Unit,COGS,Other Expenses,Gross Profit,Net Profit
count,515900.000000,515900,515900.000000,484720.000000,515768.000000,484720.000000,477691.0,116479.000000,484720.000000,116479.000000,116479.000000
mean,64487.000000,2020-11-10 17:49:27.951928832,0.227311,568.548176,463966.236509,6.254030,inf,2.867849,1.250844,4.301499,2.867528
min,0.000000,2019-03-31 00:00:00,0.000000,0.000000,110001.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,32243.000000,2020-01-22 00:00:00,0.000000,387.000000,382421.000000,4.257000,NaN,1.960000,0.830000,2.947000,1.926000
50%,64487.000000,2020-11-13 12:00:00,0.000000,526.856601,500033.000000,5.795423,NaN,2.650000,1.130000,4.023000,2.629000
75%,96731.000000,2021-09-05 18:00:00,0.000000,695.617263,600024.000000,7.651790,NaN,3.520000,1.560000,5.261000,3.617000
max,128974.000000,2022-06-29 00:00:00,15.000000,5584.000000,989898.000000,61.424000,inf,22.950000,10.880000,38.704000,28.264000
std,37231.911568,NaN,0.425335,253.669669,191476.208071,2.790366,NaN,1.294770,0.608096,1.897028,1.336707


In [12]:
df.drop(columns=["index", "Unnamed: 22"], inplace=True)

In [13]:
df['Amount_EUR'].max()

61.424

### Agregador de Data Historica

In [14]:
min_date = df['Date'].min()
max_date = df['Date'].max()
print(f"Rango de fechas: {min_date} - {max_date}")

Rango de fechas: 2019-03-31 00:00:00 - 2022-06-29 00:00:00


### Descargar Archivo

In [15]:
df.to_csv("./sales-data/amazon-sale-report-clean.csv", index=False)